In [2]:
import pandas as pd
data=pd.read_csv('C:/Users/USER/Desktop/진행중/FEMS/datamart/HS_DataMart_v1.0.csv',encoding='cp949')

In [3]:
data=data.drop(['Unnamed: 0'], axis=1)
data['datetime']=pd.to_datetime(data['datetime'])
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4366 entries, 0 to 4365
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   datetime                         4366 non-null   datetime64[ns]
 1   초지_전력사용량                         4366 non-null   float64       
 2   Press_전력사용량                      4366 non-null   float64       
 3   Constant_전력사용량                   4366 non-null   float64       
 4   Wire_전력사용량                       4366 non-null   float64       
 5   AIR_전력사용량                        4366 non-null   float64       
 6   Size_전력사용량                       4366 non-null   float64       
 7   Calender_전력사용량                   4366 non-null   float64       
 8   Drive_전력사용량                      4366 non-null   float64       
 9   Pre-Dryer_전력사용량                  4366 non-null   float64       
 10  After-Dryer_전력사용량                4366 non-null   float64    

In [4]:
#train,test split

from math import floor
def train_test_split_df(df, test_size):
  # TODO: test_size(0 < test_size < 1) 비율에 따라 테스트/학습 데이터프레임으로 나누어 반환
  div = floor(df.shape[0] * (1 - test_size))
  return df.loc[:div], df.loc[div + 1:]

train_df, test_df = train_test_split_df(data, 0.20)
print('Train data :',train_df['datetime'].min(),' ~ ',train_df['datetime'].max(),'(',len(train_df),')')
print('Test data :',test_df['datetime'].min(),' ~ ',test_df['datetime'].max(),'(',len(test_df),')')


Train data : 2022-04-26 00:00:00  ~  2022-06-01 09:00:00 ( 3493 )
Test data : 2022-06-01 09:15:00  ~  2022-06-10 11:15:00 ( 873 )


In [5]:
#1.  (비정상) column downtime 제거   
# '(비정상)지절시 운전' : 1을 제거
# '(비정상)카렌더 SHEET BREAK SENSOR TAG' : 1을 제거
# '(비정상)PICK-UP VACCUM 파괴 밸브 TAG' : 0을 제거
train_df['(비정상)지절시 운전']=train_df['(비정상)지절시 운전'].astype(str)
train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG']=train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'].astype(str)
train_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG']=train_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG'].astype(str)

train_df=train_df[train_df['(비정상)지절시 운전'] == '0.0']   
train_df=train_df[train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'] == '0.0']
train_df=train_df[train_df['(비정상)PICK-UP VACCUM 파괴 밸브 TAG'] == '1.0']
train_df.info()
print('Train data :',train_df['datetime'].min(),' ~ ',train_df['datetime'].max(),'(',len(train_df),')')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3349 entries, 0 to 3492
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   datetime                         3349 non-null   datetime64[ns]
 1   초지_전력사용량                         3349 non-null   float64       
 2   Press_전력사용량                      3349 non-null   float64       
 3   Constant_전력사용량                   3349 non-null   float64       
 4   Wire_전력사용량                       3349 non-null   float64       
 5   AIR_전력사용량                        3349 non-null   float64       
 6   Size_전력사용량                       3349 non-null   float64       
 7   Calender_전력사용량                   3349 non-null   float64       
 8   Drive_전력사용량                      3349 non-null   float64       
 9   Pre-Dryer_전력사용량                  3349 non-null   float64       
 10  After-Dryer_전력사용량                3349 non-null   float64    

C:\Users\USER\AppData\Local\Temp\ipykernel_3108\4079308839.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['(비정상)지절시 운전']=train_df['(비정상)지절시 운전'].astype(str)
C:\Users\USER\AppData\Local\Temp\ipykernel_3108\4079308839.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG']=train_df['(비정상)카렌더 SHEET BREAK SENSOR TAG'].astype(str)
C:\Users\USER\AppData\Local\Temp\ipykernel_3108\4079308839.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [6]:
train_features=train_df[['datetime',
'TURNING ROLL 보류율',
'사이로 백수 온도',
'TURNING ROLL_WIRE 진공도 토탈',
'TURNING ROLL 속도',
 'CENTER NO2 ROLL 속도',
 '1군 DRYER 속도',
 'NO8 DRYER 속도',
 'CALENDER BTM ROLL속도',
 'PM23_평량',
 'PM23_릴속도']]
train_target=train_df['초지_전력사용량']

test_features=test_df[['datetime', 
'TURNING ROLL 보류율',
'사이로 백수 온도',
'TURNING ROLL_WIRE 진공도 토탈',
'TURNING ROLL 속도',
 'CENTER NO2 ROLL 속도',
 '1군 DRYER 속도',
 'NO8 DRYER 속도',
 'CALENDER BTM ROLL속도',
 'PM23_평량',
 'PM23_릴속도']]
test_target=test_df['초지_전력사용량']

pycaret(auto ml)


In [9]:
from pycaret.regression import *

ModuleNotFoundError: No module named 'pycaret'

catboost

In [7]:
import catboost
# Initialize CatBoostRegressor
model = catboost.CatBoostRegressor(iterations=2,
                          learning_rate=0.1,
                          depth=2)
# Fit model
model.fit(train_features, train_target)
# Get predictions
preds = model.predict(test_features)

0:	learn: 121.0317286	total: 151ms	remaining: 151ms
1:	learn: 120.1401552	total: 152ms	remaining: 0us


In [ ]:
import optuna

def objective(trial):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 0.3),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 1, 15),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
    }
    

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_uniform("subsample", 0.1, 1)

    gbm = catboost.CatBoostClassifier(**param, iterations = 10000)

    gbm.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 0, early_stopping_rounds = 100)

    preds = gbm.predict(X_val)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_val, pred_labels)
    
    return accuracy

study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 200, show_progress_bar = True)

In [12]:
len(preds)


873

hyperparameter tunning(optuna)

In [ ]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.metrics import mean_absolute_error